In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # safe default
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling

dataset = load_dataset("json", data_files="cleaned_dataset.json")
tokenized_dataset = dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=512), batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/89616 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="phi2-lora-output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=1,
    max_steps=100, # time
    save_steps=100,
    save_total_limit=2,
    learning_rate=2e-4,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


<ipython-input-8-c9d6731ac462>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Train
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.674900
20,1.661800
30,1.636000
40,1.616600
50,1.659900
60,1.624800
70,1.565600
80,1.692500
90,1.660300
100,1.660900


TrainOutput(global_step=100, training_loss=1.645331516265869, metrics={'train_runtime': 432.1055, 'train_samples_per_second': 0.926, 'train_steps_per_second': 0.231, 'total_flos': 3257835454464000.0, 'train_loss': 1.645331516265869, 'epoch': 0.004463488662738797})

In [ ]:
!zip -r phi2-lora-checkpoint.zip phi2-lora-output/checkpoint-100

  adding: phi2-lora-output/checkpoint-100/ (stored 0%)
  adding: phi2-lora-output/checkpoint-100/added_tokens.json (deflated 84%)
  adding: phi2-lora-output/checkpoint-100/optimizer.pt (deflated 8%)
  adding: phi2-lora-output/checkpoint-100/tokenizer_config.json (deflated 94%)
  adding: phi2-lora-output/checkpoint-100/adapter_model.safetensors (deflated 8%)
  adding: phi2-lora-output/checkpoint-100/training_args.bin (deflated 51%)
  adding: phi2-lora-output/checkpoint-100/rng_state.pth (deflated 25%)
  adding: phi2-lora-output/checkpoint-100/scheduler.pt (deflated 57%)
  adding: phi2-lora-output/checkpoint-100/special_tokens_map.json (deflated 75%)
  adding: phi2-lora-output/checkpoint-100/scaler.pt (deflated 60%)
  adding: phi2-lora-output/checkpoint-100/trainer_state.json (deflated 69%)
  adding: phi2-lora-output/checkpoint-100/adapter_config.json (deflated 54%)
  adding: phi2-lora-output/checkpoint-100/README.md (deflated 66%)
  adding: phi2-lora-output/checkpoint-100/tokenizer.json

In [ ]:
from google.colab import files
files.download("phi2-lora-checkpoint.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>